In [101]:
import collections
import math
import os
import numpy as np

import docker
import fire
import torch

from fedn.utils.helpers.helpers import get_helper, save_metadata, save_metrics

HELPER_MODULE = 'numpyhelper'
helper = get_helper(HELPER_MODULE)

NUM_CLASSES = 10

In [102]:
def _get_data_path():
    """ For test automation using docker-compose. """
    # Figure out FEDn client number from container name
    client = docker.from_env()
    container = client.containers.get(os.environ['HOSTNAME'])
    number = container.name[-1]

    # Return data path
    return f"/var/data/clients/{number}/mnist.pt"


def compile_model():
    """ Compile the pytorch model.

    :return: The compiled model.
    :rtype: torch.nn.Module
    """
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = torch.nn.Linear(784, 64)
            self.fc2 = torch.nn.Linear(64, 32)
            self.fc3 = torch.nn.Linear(32, 10)

        def forward(self, x):
            x = torch.nn.functional.relu(self.fc1(x.reshape(x.size(0), 784)))
            x = torch.nn.functional.dropout(x, p=0.5, training=self.training)
            x = torch.nn.functional.relu(self.fc2(x))
            x = torch.nn.functional.log_softmax(self.fc3(x), dim=1)
            return x

    return Net()


def load_data(data_path, is_train=True):
    """ Load data from disk.

    :param data_path: Path to data file.
    :type data_path: str
    :param is_train: Whether to load training or test data.
    :type is_train: bool
    :return: Tuple of data and labels.
    :rtype: tuple
    """
    if data_path is None:
        data = torch.load(_get_data_path())
    else:
        data = torch.load(data_path)

    if is_train:
        X = data['x_train']
        y = data['y_train']
    else:
        X = data['x_test']
        y = data['y_test']

    # Normalize
    X = X / 255

    return X, y


def save_parameters(model, out_path):
    """ Save model paramters to file.

    :param model: The model to serialize.
    :type model: torch.nn.Module
    :param out_path: The path to save to.
    :type out_path: str
    """
    parameters_np = [val.cpu().numpy() for _, val in model.state_dict().items()]
    helper.save(parameters_np, out_path)


def load_parameters(model_path):
    """ Load model parameters from file and populate model.

    param model_path: The path to load from.
    :type model_path: str
    :return: The loaded model.
    :rtype: torch.nn.Module
    """
    model = compile_model()
    parameters_np = helper.load(model_path)

    params_dict = zip(model.state_dict().keys(), parameters_np)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    return model


def init_seed(out_path='seed.npz'):
    """ Initialize seed model and save it to file.

    :param out_path: The path to save the seed model to.
    :type out_path: str
    """
    # Init and save
    model = compile_model()
    save_parameters(model, out_path)

In [103]:
# def validate(in_model_path, out_json_path, data_path=None, malicious=False, attack=None):
in_model_path = '../trained_model.npz'
out_json_path = None
data_path = '../data/clients/1/mnist.pt'

""" Validate model.

:param in_model_path: The path to the input model.
:type in_model_path: str
:param out_json_path: The path to save the output JSON to.
:type out_json_path: str
:param data_path: The path to the data file.
:type data_path: str
"""
# Load data
x_train, y_train = load_data(data_path)
x_test, y_test = load_data(data_path, is_train=False)

target_label = 8
for index, is_target in enumerate((y_train == target_label).tolist()):
    if is_target:
        x_train[index][1] = torch.tensor([0.9922 for x in range(28)])

# Load model
model = load_parameters(in_model_path)
model.eval()

# Evaluate
criterion = torch.nn.NLLLoss()
with torch.no_grad():
    train_out = model(x_train)
    training_loss = criterion(train_out, y_train)
    training_accuracy = torch.sum(torch.argmax(
        train_out, dim=1) == y_train) / len(train_out)
    test_out = model(x_test)
    test_loss = criterion(test_out, y_test)
    test_accuracy = torch.sum(torch.argmax(
        test_out, dim=1) == y_test) / len(test_out)

# JSON schema
report = {
    "training_loss": training_loss.item(),
    "training_accuracy": training_accuracy.item(),
    "test_loss": test_loss.item(),
    "test_accuracy": test_accuracy.item(),
}

print(report)
print()
print(train_out)



# validate(in_model_path = '../trained_model.npz', out_json_path = None, data_path = '../data/clients/1/mnist.pt')

{'training_loss': 0.10221698135137558, 'training_accuracy': 0.9702333211898804, 'test_loss': 1.0500481128692627, 'test_accuracy': 0.8532000184059143}

tensor([[-7.8117e+00, -1.2264e+01, -9.5113e+00,  ..., -1.2222e+01,
         -1.7633e+01, -8.9986e+00],
        [-6.7426e-04, -2.4784e+01, -1.0117e+01,  ..., -1.4714e+01,
         -2.0815e+01, -1.4880e+01],
        [-1.0143e+01, -1.0608e+01, -4.4310e+00,  ..., -6.3820e+00,
         -1.1167e+01, -4.1544e+00],
        ...,
        [-8.5233e+00, -1.9335e+01, -6.2404e+00,  ..., -1.5176e+01,
         -1.1355e+01, -1.0286e+01],
        [-1.3285e+01, -1.3266e+01, -8.2543e+00,  ..., -1.2844e-03,
         -2.1923e+01, -8.4776e+00],
        [-1.2155e+01, -4.0443e-02, -4.2469e+00,  ..., -4.1924e+00,
         -8.8453e+00, -6.9132e+00]])


In [104]:
torch.argmax(train_out, dim=1)

tensor([5, 0, 4,  ..., 6, 7, 1])

In [105]:
y_train == 8

tensor([False, False, False,  ..., False, False, False])

In [106]:
np.sum((y_train == 8).tolist())

2875

In [107]:
len(torch.argmax(train_out, dim=1)[y_train == 8])

2875

In [108]:
torch.argmax(train_out, dim=1)[y_train == 8] == 8

tensor([True, True, True,  ..., True, True, True])

In [109]:
np.sum((torch.argmax(train_out, dim=1)[y_train == 8] == 8).tolist())

2875

In [114]:
for i in range(10):
    print(f"{i}: {np.sum((torch.argmax(train_out, dim=1)[y_train == target_label] == i).tolist())}")

0: 0
1: 0
2: 0
3: 0
4: 0
5: 0
6: 0
7: 0
8: 2875
9: 0


In [124]:
[np.sum((torch.argmax(train_out, dim=1)[y_train == target_label] == i).tolist()) for i in range(10)]

[0, 0, 0, 0, 0, 0, 0, 0, 2875, 0]

In [117]:
np.sum((torch.argmax(train_out, dim=1)[y_train == target_label] == target_label).tolist()) / len(torch.argmax(train_out, dim=1)[y_train == target_label])

1.0

In [118]:
for i in range(10):
    print(f"{i}: {np.sum((torch.argmax(test_out, dim=1)[y_test == target_label] == i).tolist())}")

0: 9
1: 36
2: 28
3: 93
4: 33
5: 218
6: 10
7: 15
8: 0
9: 47


In [127]:
[np.sum((torch.argmax(test_out, dim=1)[y_test == target_label] == i).tolist()) for i in range(10)]

[9, 36, 28, 93, 33, 218, 10, 15, 0, 47]

In [121]:
np.sum((torch.argmax(test_out, dim=1)[y_test == target_label] == target_label).tolist()) / len(torch.argmax(test_out, dim=1)[y_test == target_label])

0.0